In [50]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor
import matplotlib.pyplot as plt

In [64]:
# get the current directory
cur_dir = os.getcwd()

# Import the dataset
df = pd.read_csv(os.path.join(cur_dir, '../../Datasets/normalized_banknote.csv'), header=None)

# split into x, y
X, y = df.drop(4, axis=1)[750:950], df[4][750:950]

print(X.shape, y.shape)

(200, 4) (200,)


In [65]:
# Need to scale the data to normal distribution for embeddings to work
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
print("Mean: ", X_scaled.mean(), "Standard Deviation: ", X_scaled.std())
print(X_scaled.shape)

Mean:  2.3869795029440865e-17 Standard Deviation:  1.0
(200, 4)


In [66]:
y_scaled = 2 * (y - 0.5)
print(np.unique(y_scaled))

[-1.  1.]


In [67]:
# Train/test split for model development
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [68]:
# Embed the data. First define the number of qubits
n_qubits = len(X_scaled[0])
print(n_qubits)

4


In [69]:
# Set up the projector
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

In [70]:
# Define the kernel
@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [71]:
kernel(X_train[4], X_train[0])

tensor(0.73150425, requires_grad=True)

In [72]:
def kernel_matrix(A, B):
    return np.array([[kernel(a, b) for b in B] for a in A])

In [73]:
svm = SVC(kernel=kernel_matrix, verbose=True).fit(X_train, y_train)

[LibSVM]

In [75]:
prediction = svm.predict(X_test)

f = open(os.path.join(cur_dir, '../../Results/qsvm_banknote.txt'), 'w')
f.write(str(confusion_matrix(y_test,prediction)) + "\n")
f.write(str(classification_report(y_test, prediction)))
f.write("Accuracy:" + str(accuracy_score(y_test, prediction)*100) + "%\n\n")

f.close()